In [7]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as copt

In [4]:
from google.colab import drive

In [5]:
data = np.loadtxt('/content/drive/MyDrive/PRNN/Assignment_2/multi_class/multi_class_classification_data_group_5_train.txt', delimiter='\t',skiprows=1)
print(data.shape)

(70000, 26)


Splitting Data

In [19]:
train_ratio,test_ratio = 0.01,0.99

np.random.shuffle(data)

num_samples = len(data)
num_train,num_test = int(train_ratio * num_samples),int(test_ratio * num_samples)
#split data
train_data,test_data = data[:num_train],data[num_train:]

print("Training set size:", len(train_data))
print("Test set size:", len(test_data))

X_train = train_data[:, :-1]  # Features
y_train = train_data[:, -1]   # Labels
X_test = test_data[:, :-1]  # Features
y_test = test_data[:, -1]   # Labels
float_array = np.array(y_test)
y_test = float_array.astype(int)
num_classes = 10


Training set size: 700
Test set size: 69300


Define Kernels

In [20]:
def linear_kernel(X1, X2):
    return np.dot(X1, X2.T)


In [ ]:
def polynomial_kernel(X1, X2, degree=3):
    return (np.dot(X1, X2.T) + 1) ** degree

In [ ]:
def rbf_kernel(X1, X2, gamma=1.0):
    n1 = np.shape(X1)[0]
    n2 = np.shape(X2)[0]
    K = np.zeros((n1, n2))
    for i in range(n1):
        for j in range(n2):
            K[i,j] = np.exp(-gamma * np.linalg.norm(X1[i] - X2[j])**2)
    return K


Define Optimization function

In [21]:
def optimize_dual(X, y, kernel, C):
    n_samples, n_features = X.shape

    # Compute the Gram matrix
    K = kernel(X, X)

    # Define the quadratic and linear terms of the QP problem
    P = copt.matrix(np.outer(y, y) * K)
    q = copt.matrix(-np.ones(n_samples))
    G = copt.matrix(np.vstack((-np.eye(n_samples), np.eye(n_samples))))
    h = copt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    A = copt.matrix(y.astype(float), (1, n_samples))
    b = copt.matrix(0.0)
    # Solve the QP problem
    # cvxopt.solvers.options['show_progress']=False
    sol = copt.solvers.qp(P, q, G, h, A, b)

    # Extract lagrange multipliers
    alpha = np.array(sol['x'])
    return alpha

Training Function for OneVsRest

In [22]:
def train_svm_one_vs_rest(X_train, y_train, kernel, C):
    n_samples, n_features = X_train.shape
    n_classes = len(np.unique(y_train))
    models = []

    '''for i in range(n_classes):
        # Convert the problem to binary classification: class i vs rest
        binary_y_train = np.where(y_train == i, 1, -1)
        alpha = optimize_dual(X_train, binary_y_train, kernel, C)

        # Compute support vectors
        sv_idx = alpha > 0
        sv_idx = sv_idx.flatten()
        support_vectors = X_train[sv_idx]
        support_vector_labels = binary_y_train[sv_idx]
        alpha_sv = alpha[sv_idx]

        # Compute bias term
        kernel_matrix = kernel(support_vectors, support_vectors)
        alpha_sv = alpha_sv.reshape(-1,)
        product = (support_vector_labels * alpha_sv)
        decision_values = np.dot(kernel_matrix, product)
        bias = np.mean(support_vector_labels - decision_values)

        # Store the trained model
        models.append({'class': i, 'support_vectors': support_vectors, 'support_vector_labels': support_vector_labels,
                       'alpha_sv': alpha_sv, 'bias': bias})
        return models'''

Predict Function

In [23]:
def predict_svm_one_vs_rest(X_test, models, kernel):
    n_samples_test = X_test.shape[0]
    n_classes = len(models)
    decision_functions = np.zeros((n_samples_test, n_classes))

    for i, model in enumerate(models):
        decision_function = np.dot(kernel(X_test, model['support_vectors']), (model['support_vector_labels'] * model['alpha_sv'])) + model['bias']
        decision_functions[:, i] = decision_function

    predicted_labels = np.argmax(decision_functions, axis=1)
    return predicted_labels

In [24]:
models = train_svm_one_vs_rest(X_train, y_train, polynomial_kernel, C=1.0)

# Predict using OvR SVM models
y_pred = predict_svm_one_vs_rest(X_test, models, polynomial_kernel)

# Evaluate accuracy or other metrics
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

     pcost       dcost       gap    pres   dres
 0: -1.5164e-03 -7.0143e+02  3e+03  1e+00  2e-14
 1:  1.0738e-01 -1.8371e+02  2e+02  2e-02  2e-14
 2:  4.9218e-02 -1.8945e+01  2e+01  2e-03  2e-14
 3:  3.8288e-02 -6.0775e+00  7e+00  5e-04  2e-14
 4:  3.3662e-02 -2.0320e+00  2e+00  1e-04  1e-14
 5: -2.5021e-02 -1.0272e-01  8e-02  1e-06  8e-15
 6: -3.7750e-02 -5.9593e-02  2e-02  3e-07  5e-15
 7: -4.1524e-02 -4.8468e-02  7e-03  8e-08  5e-15
 8: -4.3120e-02 -4.4761e-02  2e-03  2e-16  5e-15
 9: -4.3480e-02 -4.3643e-02  2e-04  2e-16  4e-15
10: -4.3525e-02 -4.3531e-02  5e-06  2e-16  5e-15
11: -4.3527e-02 -4.3527e-02  2e-07  2e-16  5e-15
12: -4.3527e-02 -4.3527e-02  7e-09  2e-16  5e-15
Optimal solution found.
Accuracy: 0.09968253968253968


In [26]:
#let us define optimization function without regularization Parameter C
'''def optimize_dual_C(X, y, kernel):
    n_samples, n_features = X.shape

    # Compute the Gram matrix
    K = kernel(X, X)

    # Define the quadratic and linear terms of the QP problem
    P = copt.matrix(np.outer(y, y) * K)
    q = copt.matrix(-np.ones(n_samples))
    G = copt.matrix(-np.eye(n_samples))  # Removed positive identity matrix
    h = copt.matrix(np.zeros(n_samples))  # Removed C values
    A = copt.matrix(y.astype(float), (1, n_samples))
    b = copt.matrix(0.0)
    # Solve the QP problem
    solution = copt.solvers.qp(P, q, G, h, A, b)

    # Extract lagrange multipliers
    alpha = np.array(solution['x'])
    return alpha'''

In [30]:
def train_svm_one_vs_rest_C(X_train, y_train, kernel):
    n_samples, n_features = X_train.shape
    n_classes = len(np.unique(y_train))
    models = []

    for i in range(n_classes):
        # Convert the problem to binary classification: class i vs rest
        binary_y_train = np.where(y_train == i, 1, -1)
        alpha = optimize_dual_C(X_train, binary_y_train, kernel)

        # Compute support vectors
        sv_idx = alpha > 0
        sv_idx = sv_idx.flatten()
        support_vectors = X_train[sv_idx]
        support_vector_labels = binary_y_train[sv_idx]
        alpha_sv = alpha[sv_idx]

        # Compute bias term
        kernel_matrix = kernel(support_vectors, support_vectors)
        alpha_sv = alpha_sv.reshape(-1,)
        product = (support_vector_labels * alpha_sv)
        decision_values = np.dot(kernel_matrix, product)
        bias = np.mean(support_vector_labels - decision_values)

        # Store the trained model
        models.append({'class': i, 'support_vectors': support_vectors, 'support_vector_labels': support_vector_labels,
                       'alpha_sv': alpha_sv, 'bias': bias})
        return models

In [31]:
#Predict Function
def predict_svm_one_vs_rest_C(X_test, models, kernel):
    n_samples_test = X_test.shape[0]
    n_classes = len(models)
    decision_functions = np.zeros((n_samples_test, n_classes))

    for i, model in enumerate(models):
        decision_function = np.dot(kernel(X_test, model['support_vectors']), (model['support_vector_labels'] * model['alpha_sv'])) + model['bias']
        decision_functions[:, i] = decision_function

    predicted_labels = np.argmax(decision_functions, axis=1)
    return predicted_labels

In [32]:
models = train_svm_one_vs_rest_C(X_train, y_train, polynomial_kernel)

# Predict using OvR SVM models
y_pred = predict_svm_one_vs_rest_C(X_test, models, polynomial_kernel)

# Evaluate accuracy or other metrics
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

     pcost       dcost       gap    pres   dres
 0: -5.6821e-02 -1.7153e-01  7e+02  3e+01  1e+00
 1: -1.3367e-03 -2.2721e-01  8e+00  3e-01  1e-02
 2: -2.6667e-02 -1.6661e-01  9e-01  3e-02  1e-03
 3: -4.0000e-02 -1.0883e-01  2e-01  6e-03  2e-04
 4: -3.7296e-02 -8.1088e-02  1e-01  2e-03  7e-05
 5: -3.9587e-02 -5.2468e-02  1e-02  6e-05  2e-06
 6: -4.2130e-02 -4.5859e-02  4e-03  1e-05  5e-07
 7: -4.3181e-02 -4.3886e-02  7e-04  5e-18  5e-15
 8: -4.3490e-02 -4.3559e-02  7e-05  7e-18  4e-15
 9: -4.3526e-02 -4.3528e-02  3e-06  6e-18  5e-15
10: -4.3527e-02 -4.3527e-02  1e-07  8e-18  5e-15
11: -4.3527e-02 -4.3527e-02  5e-09  1e-18  5e-15
Optimal solution found.
Accuracy: 0.09968253968253968
